In [1]:
!pip install "pymilvus[model]" openai requests tqdm torch

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [2]:
import os
api_key = os.getenv('DEEPSEEK_API_KEY')

In [3]:
from glob import glob
filepath = './mfd.md'
text_lines = []

for i in glob(filepath, recursive=True):
    with open(i,'r') as f:
        file_text =f.read()
        #for l in f:
        #    if l.strip() !='':
        #        text_lines.append(l)
    text_lines += file_text.split('# ')
print(text_lines)
print(len(text_lines))

['#', '中华人民共和国民法典\n\n##', '（二）物权编\n\n###', '第一章 一般规定\n\n**第二百零四条** 为了明确物的归属，充分发挥物的效用，保护权利人的合法权益，维护社会经济秩序，制定本编。\n\n**第二百零五条** 本编调整因物的归属和利用产生的民事关系。\n\n**第二百零六条** 国家坚持和完善社会主义公有制为主体、多种所有制经济共同发展的基本经济制度。\n国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展。\n国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发展权利。\n\n**第二百零七条** 国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯。\n\n**第二百零八条** 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。\n\n**第二百零九条** 不动产物权的设立、变更、转让和消灭，经依法登记，发生效力；未经登记，不发生效力，但是法律另有规定的除外。\n依法属于国家所有的自然资源，所有权可以不登记。\n\n**第二百一十条** 不动产登记，由不动产所在地的登记机构办理。\n国家对不动产实行统一登记制度。统一登记的范围、登记机构和登记办法，由法律、行政法规规定。\n\n**第二百一十一条** 当事人申请登记，应当根据不同登记事项提供材料。\n申请登记材料以及登记事项相关信息，可以公开查询。\n\n**第二百一十二条** 登记机构应当履行下列职责：\n（一）审查申请人提供的材料；\n（二）询问申请人；\n（三）如实、及时登记；\n（四）法律、行政法规规定的其他职责。\n申请登记的不动产存在尚未解决的权属争议的，登记机构应当不予登记，并书面告知申请人。\n\n**第二百一十三条** 登记机构不得有下列行为：\n（一）要求对不动产进行评估；\n（二）以不动产登记为条件收取其他费用；\n（三）超出登记职责范围的其他行为。\n\n**第二百一十四条** 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。\n\n**第二百一十五条** 不动产登记簿由登记机构管理。\n不动产登记簿应当采用纸质形式或者电子形式。\n不动产登记簿采用电子形式的，应当备份。\n\n**第二百一十六条** 不动产登记

In [13]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",
)

In [14]:
#定义一个 embedding 模型，使用 milvus_model 来生成文本嵌入。我们以 DefaultEmbeddingFunction 模型为例，这是一个预训练的轻量级嵌入模型。

In [15]:
from pymilvus import model as milvus_model
embedding_model = milvus_model.DefaultEmbeddingFunction()


In [16]:
#测试数据，生成embedding,打印纬度数和前几个纬度的数值

In [17]:
test_embedding = embedding_model.encode_queries(["This is a hahah"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[ 0.01634293  0.03422707 -0.00944707 -0.058086    0.01152717 -0.00139032
  0.0201766  -0.00700357 -0.01878644 -0.01590427]


In [18]:
#创建向量数据库，创建 Collection，将数据加载到 Milvus,

In [19]:
from pymilvus import MilvusClient
milvus_client = MilvusClient(uri="./milvus_mfd.db")
collection_name = "mfd_collection"

#如果库已存在就删除
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [20]:
#创建一个具有指定参数的新 collection。
#如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 id 字段作为主键，以及一个 vector 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。

In [21]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。
)

In [22]:
#插入数据

In [23]:
from tqdm import tqdm
data = []
doc_embeddings = embedding_model.encode_documents(text_lines)
doc_embeddings
for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})
    
milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 30/30 [00:00<00:00, 168445.94it/s]


{'insert_count': 30, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'cost': 0}

In [24]:
#检索查询数据

In [25]:
question_list = ["说明一下不动产的相关规定","征集土地时应该怎么做","哪些资源属于国家所有"]
question_answer_dict =dict()
for question in question_list:
    search_res = milvus_client.search(
        collection_name=collection_name,
        data = embedding_model.encode_queries([question]),  # 将问题转换为嵌入向量
        limit=3,  # 返回前3个结果
        search_params={"metric_type": "IP", "params": {}},  # 内积距离
        output_fields=["text"],  # 返回 text 字段
    )
    
    retrieved_lines_with_distances = [
        (res["entity"]["text"], res["distance"]) for res in search_res[0]
    ]
    
    import json
    print(json.dumps(retrieved_lines_with_distances, indent=4))
    #使用 LLM 获取 RAG 响应
    #将检索到的文档转换为字符串格式。   
    context = "\n".join(
        [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
    )
    context
    question_answer_dict[question] = context
    

[
    [
        "\u7b2c\u4e09\u7ae0 \u5408\u540c\u7684\u53d8\u66f4\u548c\u8f6c\u8ba9\n\n**\u7b2c\u4e94\u767e\u56db\u5341\u516b\u6761** \u5f53\u4e8b\u4eba\u534f\u5546\u4e00\u81f4\uff0c\u53ef\u4ee5\u53d8\u66f4\u5408\u540c\u3002\n\n**\u7b2c\u4e94\u767e\u56db\u5341\u4e5d\u6761** \u5f53\u4e8b\u4eba\u5bf9\u5408\u540c\u53d8\u66f4\u7684\u5185\u5bb9\u7ea6\u5b9a\u4e0d\u660e\u786e\u7684\uff0c\u63a8\u5b9a\u4e3a\u672a\u53d8\u66f4\u3002\n\n**\u7b2c\u4e94\u767e\u4e94\u5341\u6761** \u503a\u6743\u4eba\u53ef\u4ee5\u5c06\u5408\u540c\u7684\u6743\u5229\u5168\u90e8\u6216\u8005\u90e8\u5206\u8f6c\u8ba9\u7ed9\u7b2c\u4e09\u4eba\uff0c\u4f46\u662f\u6709\u4e0b\u5217\u60c5\u5f62\u4e4b\u4e00\u7684\u9664\u5916\uff1a\n\uff08\u4e00\uff09\u6839\u636e\u5408\u540c\u6027\u8d28\u4e0d\u5f97\u8f6c\u8ba9\uff1b\n\uff08\u4e8c\uff09\u6309\u7167\u5f53\u4e8b\u4eba\u7ea6\u5b9a\u4e0d\u5f97\u8f6c\u8ba9\uff1b\n\uff08\u4e09\uff09\u4f9d\u7167\u6cd5\u5f8b\u89c4\u5b9a\u4e0d\u5f97\u8f6c\u8ba9\u3002\n\u503a\u6743\u4eba\u8f6c\u8ba9\u6743\u522

In [26]:
for question in question_answer_dict:
    context = question_answer_dict[question]
    SYSTEM_PROMPT = """
Human: 请你从提供的上下文段落片段中找到问题的答案。
"""
    USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。请在头部追加问题
<context>
{context}
</context>
<question>
{question}
</question>
"""
    response = deepseek_client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": USER_PROMPT},
        ],
    )
    print("***********************************************")
    print(response.choices[0].message.content)

***********************************************
问题：说明一下不动产的相关规定

根据提供的上下文，不动产的相关规定主要涉及地役权的内容，具体如下：

1. **地役权的定义与设立（第三百八十八条）**  
   地役权人有权依照合同约定，利用他人的不动产（供役地）以提高自己的不动产（需役地）的效益。设立地役权需通过合同约定，并依法办理登记（第三百九十条），未经登记不得对抗善意第三人。

2. **地役权合同内容（第三百八十九条）**  
   合同一般包括当事人的信息、供役地和需役地的位置、地役权的目的和期限、利用方式、报酬及支付方式、争议解决方式等条款。

3. **地役权人的权利与义务（第三百九十一条、第三百九十四条）**  
   - 权利：有权利用供役地以提高需役地的效益，但不得损害供役地权利人的合法权益。  
   - 义务：需按合同约定支付报酬，并以合理方式利用供役地。

4. **供役地权利人的义务（第三百九十二条）**  
   供役地权利人应按约定允许地役权人利用土地，且不得妨碍其行使权利。

5. **地役权的转让与抵押（第三百九十五条）**  
   地役权不得单独转让或抵押；但若土地承包经营权、建设用地使用权、宅基地使用权等权利转让或抵押时，地役权一并转让或抵押。

6. **地役权的存续与消灭（第三百九十三条、第三百九十六条）**  
   - 地役权期限届满或地役权人放弃权利时，地役权消灭，需办理注销登记。  
   - 供役地转让、出租或抵押不影响地役权的存续（第三百九十六条）。若因此导致地役权消灭，地役权人有权请求损害赔偿（第三百九十七条）。

7. **不动产的占有（第四百七十一条至第四百八十条）**  
   占有指对物事实上的控制和支配，分为直接占有和间接占有。占有人需承担占有物毁损、灭失的赔偿责任（恶意占有时），但善意占有人除外。占有被侵夺时，占有人有权请求返还原物或赔偿损失。

以上规定综合了地役权设立、行使、转让及占有等相关内容，均围绕不动产的利用与权益保护展开。
***********************************************
问题：征集土地时应该怎么做

根据提供的上下文，特别是关于地役权的相关条款（第三百八十八条至第三百九十八条），征集土